In [1]:
from skimage import io
from os import path, listdir
import numpy as np
import tifffile
import numpy as np

In [2]:
dir_path = "/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/"
ext = ".tif"
suff = "_translated"

In [3]:
def calc_adj_translation(max_tr, curr_trans, direction):
    return int(max(0, (max(0, max_tr * direction) + max_tr-curr_trans)))

In [47]:
def find_dims_post_alignment(orig_dim1, orig_dim2, translation_arr):
    max_trans = int(max(translation_arr, key = abs))
    if (np.sign(min(translation_arr)) * np.sign(max(translation_arr)) == -1):
        return max_trans, int(min(orig_dim1, orig_dim2) - (max(translation_arr) - min(translation_arr)))
    return max_trans, min(orig_dim1, orig_dim2) - abs(max_trans)

In [90]:
def translate_stack(orig_img, optimal_translations):
    substack_size = orig_img.shape[0]//len(optimal_translations[0])

    max_transx, difx = find_dims_post_alignment(orig_img.shape[1], orig_img.shape[1], optimal_translations[0])
    max_transy, dify = find_dims_post_alignment(orig_img.shape[2], orig_img.shape[2], optimal_translations[1])
    
    ret = np.empty((orig_img.shape[0], difx, dify))
    orig_ret = np.empty_like(ret)
    
    x0 = [calc_adj_translation(max_transx,x, -1) for x in optimal_translations[0]]
    y0 = [calc_adj_translation(max_transy,y, -1) for y in optimal_translations[1]]
    
    x0_b = max(0, max_transx)
    y0_b = max(0, max_transy)
    for idx, r in enumerate(orig_img):
        tr_idx = idx//substack_size
        if tr_idx >= len(x0):
            tr_idx = len(x0) - 1
        ret[idx] =  r[x0[tr_idx]:x0[tr_idx]+difx,y0[tr_idx]:y0[tr_idx]+dify]
        orig_ret[idx] = orig_img[idx][x0_b:x0_b+difx,y0_b:y0_b+dify]
    return ret, orig_ret

In [91]:
def prepare_for_tif_save(image):
    newimg = np.empty((image.shape[0], 1, image.shape[1], image.shape[2]))
    for idx, im in enumerate(image):
        newimg[idx] = np.array([im])
    return newimg.astype("uint8")

In [92]:
def save_single_img(img_path, image):
    if image is not None:
        image = prepare_for_tif_save(image)
        tifffile.imwrite(img_path, image, imagej=True, metadata={'unit': 'pixels','axes': 'ZCYX'})

In [169]:
opt_transx = np.zeros(61) #1 - w prawo
opt_transy = np.zeros(61) #1 - w dół
opt_transx[45] = 0
opt_transx[46] = opt_transx[45] + 2
opt_transx[47] = opt_transx[46] + 2
opt_transx[48] = opt_transx[47] 
opt_transx[49:] = opt_transx[48] - 1

opt_transy[45] = 0
opt_transy[46] = opt_transy[45]-1
opt_transy[47] = opt_transy[46]
opt_transy[48] = opt_transy[47]
opt_transy[49:] = opt_transy[48]-1

opt_trans = [opt_transy, opt_transx]

In [170]:
opt_trans

[array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1., -2., -2., -2.,
        -2., -2., -2., -2., -2., -2., -2., -2., -2.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 4., 4., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])]

In [171]:
name_base = "m6_r2_landmark"
fname_in = dir_path + name_base + ext
fname_out = dir_path + name_base + suff + ext
fname_out_d = dir_path + name_base + suff + "dif" + ext
orig = io.imread(fname_in).astype("uint8")

rt, orig = translate_stack(orig, opt_trans)
save_single_img(fname_out, rt)

diff = rt-orig
diff[diff<0] = 0

save_single_img(fname_out_d, diff)
